## User Satisfaction Analysis

#### Import Modules

In [1]:
import pickle
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from plot_dataframe import PlotDataFrame
from utils import Util

#### Read Data

In [2]:
df_telco_user_engagement = pd.read_csv("../data/telco_user_engagement_data.csv")
df_telco_user_engagement.head()

,MSISDN/Number,cluster,xDR Sessions,Dur (ms),Total Data Volume (Bytes)
0,33601001722,1,1.0,116720140.0,8.786906e+08
1,33601001754,0,1.0,181230963.0,1.568596e+08
2,33601002511,0,1.0,134969374.0,5.959665e+08
3,33601007832,3,1.0,49878024.0,4.223207e+08
4,33601008617,2,2.0,37104453.0,1.457411e+09


In [3]:
df_telco_user_experience = pd.read_csv("../data/telco_user_experience_data.csv")
df_telco_user_experience.head()

,MSISDN/Number,cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33601001722,0,46.0,76.0,2895381.0
1,33601001754,2,31.0,99.0,7685845.5
2,33601002511,2,59.0,97.0,4150403.0
3,33601007832,0,84.0,248.0,2396.0
4,33601008617,1,119.0,43204.5,8047274.5


#### Load Models

In [7]:
with open("../models/telco_user_engagement.pkl", "rb") as f:
    model_user_engagement = pickle.load(f)

In [8]:
with open("../models/telco_user_experience.pkl", "rb") as f:
    model_user_experience = pickle.load(f)

#### Engagement Score

In [4]:
df_engagement = df_telco_user_engagement.set_index('MSISDN/Number')[
    ['xDR Sessions', 'Dur (ms)', 'Total Data Volume (Bytes)']]

In [5]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_engagement)
scaled_data
#Scaling the data

array([[-0.54866182, -0.14578281,  0.54827419],
       [-0.54866182,  0.6146225 , -1.28049119],
       [-0.54866182,  0.0693255 , -0.16800992],
       ...,
       [-0.54866182, -1.41773868,  0.13265469],
       [-0.54866182,  0.14027633, -0.46103345],
       [-0.54866182,  1.3675591 , -0.1671958 ]])

In [6]:
normalized_data = normalize(scaled_data)
normalized_data

array([[-0.69518453, -0.1847148 ,  0.69469339],
       [-0.36033465,  0.40365445, -0.84096493],
       [-0.94927152,  0.11994406, -0.29068368],
       ...,
       [-0.35954757, -0.92906865,  0.08693091],
       [-0.75133844,  0.19209465, -0.63134   ],
       [-0.36997482,  0.9221754 , -0.11274383]])

In [10]:
less_engaged_cluster = 5
distance = model_user_engagement.fit_transform(normalized_data)
distance_from_less_engaged_cluster = list(
    map(lambda x: x[less_engaged_cluster], distance))
df_telco_user_engagement['engagement_score'] = distance_from_less_engaged_cluster
df_telco_user_engagement.head()

,MSISDN/Number,cluster,xDR Sessions,Dur (ms),Total Data Volume (Bytes),engagement_score
0,33601001722,1,1.0,116720140.0,8.786906e+08,1.516102
1,33601001754,0,1.0,181230963.0,1.568596e+08,1.584497
2,33601002511,0,1.0,134969374.0,5.959665e+08,1.755110
3,33601007832,3,1.0,49878024.0,4.223207e+08,1.890540
4,33601008617,2,2.0,37104453.0,1.457411e+09,1.075597


#### Experience Score

In [11]:
df_experience = df_telco_user_experience.set_index('MSISDN/Number')[
    ['Total Avg RTT (ms)', 'Total Avg Bearer TP (kbps)', 'Total TCP Retrans. Vol (Bytes)']]

In [12]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_experience)
scaled_data
#Scaling the data

array([[-0.6592221 , -0.67835113,  0.07974981],
       [-0.93025698, -0.67723988,  1.48764748],
       [-0.4243252 , -0.67733651,  0.44859557],
       ...,
       [-0.55080815, -0.18123297, -0.70710815],
       [-0.76763605, -0.67641851, -0.6983095 ],
       [-0.40625621, -0.6804287 , -0.73574801]])

In [13]:
normalized_data = normalize(scaled_data)
normalized_data

array([[-0.69445913, -0.71461066,  0.08401263],
       [-0.49462677, -0.36009509,  0.79099677],
       [-0.46295619, -0.7390019 ,  0.48943616],
       ...,
       [-0.60233164, -0.1981858 , -0.77325221],
       [-0.61969779, -0.54605963, -0.5637318 ],
       [-0.37568797, -0.62923071, -0.68038758]])